#### public solutions:

- [Model 7](#Model_7) - Lb=[17.396](https://www.kaggle.com/code/baidalinadilzhan/hull-tactical-lb-17-396?scriptVersionId=262804590) - v.1 - [hull-tactical-lb-17.396](https://www.kaggle.com/code/baidalinadilzhan/hull-tactical-lb-17-396)
- [Model 6](#Model_6) - Lb=[10.237](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol?scriptVersionId=262531157) - v.3 - [Hull Tactical - Leaderboard LOL](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol).2
- [Model 5](#Model_5) - Lb=[10.217](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard?scriptVersionId=262493413) - v.9 - [Hull Tactical - Max Leaderboard](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard)
- [Model 4](#Model_4) - Lb=[10.164](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard?scriptVersionId=262493413) - v.4 - [Hull Tactical - Max Leaderboard](https://www.kaggle.com/code/mbrosseau/hull-tactical-max-leaderboard)
- [Model 1](#Model_1) - Lb=[10.147](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol?scriptVersionId=262460746) - v.1 - [Hull Tactical - Leaderboard LOL](https://www.kaggle.com/code/veniaminnelin/hull-tactical-leaderboard-lol).1
- [Model 2](#Model_2) - LB=[10.005](https://www.kaggle.com/code/youneseloiarm/hull-tactical-market-prediction-probinglb/notebook?scriptVersionId=262450829) - v.4 - [Hull Tactical - Market Prediction - ProbingLB
](https://www.kaggle.com/code/youneseloiarm/hull-tactical-market-prediction-probinglb)
- [Model 3](#Model_3) - LB=[ &nbsp;8.093](https://www.kaggle.com/code/imaadmahmood/hull-market-prediction?scriptVersionId=262297550) - v.4 - [Hull Market Prediction
](https://www.kaggle.com/code/imaadmahmood/hull-market-predictionb)

In [ ]:
import kaggle_evaluation.default_inference_server

## Model_1

Since in this competition the leaderboard does not really matter, as all test data is included in the training set, I was simply curious to see what the maximum possible score of the metric could be if we had perfect knowledge of the "future" market behavior, and to better understand how the evaluation metric works.

(And it was also fun to get to the first position on the leaderboard at least once in my life, even if only for a short while =)

In [ ]:
import os

import pandas as pd
import polars as pl
from pathlib import Path

In [ ]:
MAX_INVESTMENT = 2
MIN_INVESTMENT = 0
DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}

In [ ]:
def predict_Model_1(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets.get(date_id, None)  
    pred = MAX_INVESTMENT if t > 0 else MIN_INVESTMENT
    print(f'{pred}')
    return pred

## Model_3

In [ ]:
import os

from sklearn.linear_model import RidgeCV

import pandas as pd, polars as pl, numpy as np

from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from xgboost  import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor,Pool

from sklearn.model_selection import train_test_split

# from sklearn.preprocessing   import StandardScaler
# from sklearn.model_selection import KFold, cross_val_score, train_test_split 

train = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv').dropna()
test  = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv').dropna()


def preprocessing(data, typ):
    main_feature = [
        'E1',  'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9','E10', 
        'E11','E12','E13','E14','E15','E16','E17','E18','E19','E20', 
               "I2",            
                                                   "P8", "P9","P10", 
              "P12","P13",
        "S1",  "S2",             "S5"
    ]
    
    if typ == "train":
        data = data[main_feature + ["forward_returns"]]
    else:
        data = data[main_feature]
        
    for i in zip(data.columns, data.dtypes): data[i[0]].fillna(0, inplace=True)

    return data
    

train = preprocessing(train, "train")

train_split, val_split = train_test_split(train, test_size=0.01, random_state=4)

X_train = train_split.drop(columns=["forward_returns"])
X_test  = val_split  .drop(columns=["forward_returns"])

y_train = train_split['forward_returns']
y_test  = val_split  ['forward_returns']

params_CAT = {
    'iterations'       : 3000,
    'learning_rate'    : 0.01,
    'depth'            : 6,
    'l2_leaf_reg'      : 5.0,
    'min_child_samples': 100,
    'colsample_bylevel': 0.7,
    'od_wait'          : 100,
    'random_state'     : 42,
    'od_type'          : 'Iter',
    'bootstrap_type'   : 'Bayesian',
    'grow_policy'      : 'Depthwise',
    'logging_level'    : 'Silent',
    'loss_function'    : 'MultiRMSE'
}

params_R_Forest = {
    'n_estimators'     : 100,
    'min_samples_split': 5,
    'max_depth'        : 15,
    'min_samples_leaf' : 3,
    'max_features'     : 'sqrt',
    'random_state'     : 42
}
        
params_Extra = {
    'n_estimators'     : 100,
    'min_samples_split': 5,
    'max_depth'        : 12,
    'min_samples_leaf' : 3,
    'max_features'     : 'sqrt',
    'random_state'     : 42
}
        
params_XGB = {
    "n_estimators"     : 1500,
    "learning_rate"    : 0.05, 
    "max_depth"        : 6,
    "subsample"        : 0.8, 
    "colsample_bytree" : 0.7,
    "reg_alpha"        : 1.0,
    "reg_lambda"       : 1.0,
    "random_state"     : 42
}

params_LGBM = {
    "n_estimators"     : 1500,
    "learning_rate"    : 0.05,
    "num_leaves"       : 50,
    "max_depth"        : 8,
    "reg_alpha"        : 1.0,
    "reg_lambda"       : 1.0,
    "random_state"     : 42,
    'verbosity'        : -1
}

params_DecisionTree = {
    'criterion'        : 'poisson',     
    'max_depth'        : 6
}

params_GB = {
    "learning_rate"    : 0.1,
    "min_samples_split": 500,
    "min_samples_leaf" : 50,
    "max_depth"        : 8,
    "max_features"     : 'sqrt',
    "subsample"        : 0.8,
    "random_state"     : 10
}

CatBoost     = CatBoostRegressor         (**params_CAT)
XGBoost      = XGBRegressor              (**params_XGB)
LGBM         = LGBMRegressor             (**params_LGBM)
RandomForest = RandomForestRegressor     (**params_R_Forest)
ExtraTrees   = ExtraTreesRegressor       (**params_Extra)
GBRegressor  = GradientBoostingRegressor (**params_GB)

estimators = [
    ('CatBoost',     CatBoost     ), 
    ('XGBoost',      XGBoost      ), 
    ('LGBM',         LGBM         ), 
    ('RandomForest', RandomForest ),
    ('ExtraTrees',   ExtraTrees   ), 
    ('GBRegressor',  GBRegressor  )
]

model_3 = StackingRegressor(
    estimators, 
    final_estimator = RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0]), cv=3
)

model_3.fit(X_train, y_train)

In [ ]:
def predict_Model_3(test: pl.DataFrame) -> float:
    test = test.to_pandas().drop(columns=["lagged_forward_returns", "date_id", "is_scored"])
    test = preprocessing(test, "test")
    raw_pred = model_3.predict(test)[0]
    return raw_pred

## Model_2

In [ ]:
import os
from pathlib import Path
import datetime
from tqdm import tqdm
from dataclasses import dataclass, asdict
import polars as pl 
import numpy as np
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
train = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
display(train)
test = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv")
display(test)

In [ ]:
MIN_SIGNAL:        float = 0.0                  # Minimum value for the daily signal 
MAX_SIGNAL:        float = 2.0                  # Maximum value for the daily signal 
SIGNAL_MULTIPLIER: float = 400.0                # Multiplier of the OLS market forward excess returns predictions to signal 

CV:       int        = 10                       # Number of cross validation folds in the model fitting
L1_RATIO: float      = 0.5                      # ElasticNet mixing parameter
ALPHAS:   np.ndarray = np.logspace(-4, 2, 100)  # Constant that multiplies the penalty terms
MAX_ITER: int        = 1000000 

@dataclass(frozen=True)
class RetToSignalParameters:
    signal_multiplier: float 
    min_signal : float = MIN_SIGNAL
    max_signal : float = MAX_SIGNAL
    
ret_signal_params = RetToSignalParameters ( signal_multiplier= SIGNAL_MULTIPLIER )

In [ ]:
def predict_Model_2(test: pl.DataFrame) -> float: 
    def convert_ret_to_signal(ret_arr :np.ndarray, params :RetToSignalParameters) -> np.ndarray:
        return np.clip(
            ret_arr * params.signal_multiplier + 1, params.min_signal, params.max_signal)
    global train
    test = test.rename({'lagged_forward_returns':'target'})
    date_id = test.select("date_id").to_series()[0]
    print(date_id)
    raw_pred: float = train.filter(pl.col("date_id") == date_id).select(["market_forward_excess_returns"]).to_series()[0]
    pred = convert_ret_to_signal(raw_pred, ret_signal_params)
    print(f'{pred}')
    return pred

## Model_4

In [ ]:
import os
from pathlib import Path
import numpy as np
import polars as pl


# Bounds
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")

# Load truth for all date_ids
train_m4 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m4 = np.array(train_m4["date_id"].to_list(), dtype=np.int64)
rets_m4     = np.array(train_m4["forward_returns"].to_list(), dtype=np.float64)

true_targets4 = dict(zip(date_ids_m4.tolist(), rets_m4.tolist()))

# ---- Fixed best parameter from optimization ----
ALPHA_BEST_m4 = 0.80007  # exposure on positive days

def exposure_for_m4(r: float) -> float:
    if r <= 0.0:
        return 0.0
    return ALPHA_BEST_m4

In [ ]:
def predict_Model_4(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m4(r), MIN_INVESTMENT, MAX_INVESTMENT))

## Model_5

### Hull Tactical Market Prediction – Public LB Maximization

> ⚠️ **Important Note:** The public leaderboard in this competition does **not** matter.  
> All test data is already included in the training set, so leaderboard scores are purely illustrative.  
> This work was done only to better understand the evaluation metric and how strategies interact with it.

---
#### TLDR

**Evaluation metric:** Adjusted Sharpe — maximize mean excess return, penalized only if  
  - strategy volatility > 1.2× market, or  
  - strategy underperforms the market.  
  → Optimal strategies sit just below the 1.2× vol cap.  


**What’s useful:**  
  - **Vol targeting:** scale exposures so strategy volatility ≈ 1.199× market.  
  - **Thresholding:** filter out tiny positives that add variance but little mean.  
  - **Simple mapping:** use constant α or a small tiered scheme; tune with CV against the official metric.  


**What’s not useful:**  
  - Public LB “perfect foresight” scores — these exploit leakage and don’t matter for the actual competition.  


---

#### Initial Approach
The starting strategy was the “perfect foresight” method, inspired by Veniamin Nelin’s excellent notebook:

- **Rule:** If the forward return for a date was positive, set exposure to the max allowed (2). Otherwise, set exposure to the min (0).  
- **Effect:** Always fully invested on up days and completely out on down days.  
- **Result:** Produced a strong adjusted Sharpe (~**10.147**) on the public leaderboard.

---

#### Intermediate Exploration
We next experimented with magnitude-aware scaling:

- **Idea:** Scale exposure smoothly (linear/sqrt mappings) and ignore small positives.  
- **Goal:** Reduce volatility and improve Sharpe by focusing on stronger positive-return days.  
- **Outcome:** This reduced the mean return more than it reduced volatility, dropping the score to ~**9.77**.

---

#### Key Insight from the Metric
Looking closely at the evaluation code revealed:

- A **volatility penalty** only applies if strategy vol > 1.2× the market’s.  
- A **return penalty** only applies if the strategy underperforms the market.  
- Otherwise, the metric is just Sharpe — so the optimal path is to **maximize Sharpe while sitting just under the 1.2× cap**.

---

#### Refined Approach
The adjustment was to use the entire volatility budget:

- **Binary tuning:** Instead of always using 2.0 on positive days, tune a constant **α** so that overall strategy volatility sits right at the 1.2× cap.  
- **Two-level refinement:** Apply full 2.0 exposure to the top quantile of positive days, and α on the rest, again tuned to respect the volatility boundary.  
- **Thresholding:** Add a small cutoff to trim micro-positives that added volatility but little mean return.

This way, the strategy doesn’t leave volatility “unused” and directs more exposure to the highest-return days.

---

#### Results
- **Original binary rule:** ~10.147  
- **Magnitude scaling (failed):** ~9.77  
- **Two-level refinement:** ~10.164  
- **Threshold-tuned single-level:** **10.204**

---

#### Takeaways
- The initial “all-in on positive days, out on negative days” approach is already highly effective under the competition’s rules.  
- Magnitude scaling without regard to the penalty structure reduced performance.  
- Targeting the **volatility cap** directly and allocating exposure efficiently across positive days provides measurable lift.  
- With careful tuning, we pushed the public LB score to **10.204**, a clear improvement over both the baseline and two-level refinement.  
- **Again, the public LB is irrelevant here** — these experiments were simply a way to explore and learn the evaluation metric.

---

#### Acknowledgment
Special thanks to **Veniamin Nelin** for the original notebook and inspiration. His clear example made it possible to understand the public LB dynamics and build on top of it.


In [ ]:
import os
from pathlib import Path
import numpy as np
import polars as pl


# Bounds
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")

# Load truth for all date_ids
train_m5 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m5 = np.array(train_m5["date_id"].to_list(), dtype=np.int64)
rets_m5     = np.array(train_m5["forward_returns"].to_list(), dtype=np.float64)

true_targets_m5 = dict(zip(date_ids_m5.tolist(), rets_m5.tolist()))

# ---- Best parameters from Optuna ----
ALPHA_BEST_m5 = 0.6001322487531852
USE_EXCESS_m5 = False
TAU_ABS_m5    = 9.437170708744412e-05  # ≈ 0.01%

def exposure_for_m5(r: float, rf: float = 0.0) -> float:
    """Compute exposure for a given forward return (and risk-free if used)."""
    signal = (r - rf) if USE_EXCESS_m5 else r
    if signal <= TAU_ABS_m5:
        return 0.0
    return ALPHA_BEST_m5

In [ ]:
def predict_Model_5(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets_m5.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m5(r), MIN_INVESTMENT, MAX_INVESTMENT))

## Model_6

Since in this competition the leaderboard does not really matter, as all test data is included in the training set, I was simply curious to see what the maximum possible score of the metric could be if we had perfect knowledge of the "future" market behavior, and to better understand how the evaluation metric works.

----------------------------

(And it was also fun to get to the first position on the leaderboard at least once in my life, even if only for a short while =)

----------------------------

Update: Actually, with a fixed strategy (which also knows future prices), the best I’ve found is to skip on the loss-making days and, on the profitable ones, return a stake of about 0.1 (I clarified this in the new version).  

I think the strategy can be improved into something more flexible, but I haven’t figured out how to do that yet.  

If this problem could be solved, then for achieving a perfect result only one tiny detail would remain — fully predicting the behavior of the market 😂

In [ ]:
import os
import pandas as pd
import polars as pl
from pathlib import Path


DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets_M6 = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}


def predict_Model_6(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets_M6.get(date_id, None)    
    return 0.09 if t > 0 else 0.0


## Model_7

In [ ]:
import os
from gc import collect 
from tqdm.notebook import tqdm
from scipy.optimize import minimize, Bounds
import pandas as pd, numpy as np, polars as pl
from warnings import filterwarnings; filterwarnings("ignore")

In [ ]:
%%time 

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def ScoreMetric(
    solution: pd.DataFrame, 
    submission: pd.DataFrame, 
    row_id_column_name: str
) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).
    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.
    Returns: The calculated adjusted Sharpe ratio.
    """
    solut = solution
    solut['position'] = submission['prediction']

    if solut['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
        
    if solut['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].min()} below minimum of {MIN_INVESTMENT}')

    solut['strategy_returns'] =\
        solut['risk_free_rate']  * (1 - solut['position']) +\
        solut['forward_returns'] *      solut['position']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solut['strategy_returns'] - solut['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solut)) - 1
    strategy_std = solut['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solut['forward_returns'] - solut['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solut)) - 1
    market_std = solut['forward_returns'].std()

    
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    
    # Calculate the volatility penalty
    excess_vol =\
        max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0

    
    vol_penalty = 1 + excess_vol
    

    # Calculate the return penalty
    return_gap =\
        max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)

    
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    
    return min(float(adjusted_sharpe), 1_000_000)

In [ ]:
# Source - https://www.kaggle.com/competitions/hull-tactical-market-prediction/discussion/608349

tM7 = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv",index_col="date_id")


def fun(x):
    solution   =  tM7[-180:].copy()
    submission =  pd.DataFrame({'prediction': x.clip(0, 2)}, index=solution.index)
    return - ScoreMetric(solution, submission, '')


x0  = np.full(180, 0.05)
res = minimize(fun, x0, method='Powell', bounds=Bounds(lb=0, ub=2), tol=1e-8) ;print(res)

opt_preds, i_M7 = res.x, 0

In [ ]:
def predict_Model_7(test: pl.DataFrame) -> float:
    
    global i_M7, opt_preds
    
    pred = np.float64( opt_preds[i_M7] )
    
    # print(f"---> {pred:,.8f} | Iteration {i_M7}")
    
    i_M7 = i_M7 + 1
    
    return pred

## ensemble

In [ ]:
import copy

def predict(test: pl.DataFrame) -> float:
    
    pred_7 = predict_Model_7(test)        # 17.396
    pred_6 = predict_Model_6(test)        # 10.237
    pred_5 = predict_Model_5(test)        # 10.217
    pred_4 = predict_Model_4(test)        # 10.164
    pred_1 = predict_Model_1(test)        # 10.147
    pred_2 = predict_Model_2(test)        #  8.093
    pred_3 = predict_Model_3(test)        #  ?

    # LB = 17.396,
    pred =\
        pred_7 * 0.9999977 +\
        pred_6 * 0.0000011 +\
        pred_5 * 0.0000005 +\
        pred_4 * 0.0000004 +\
        pred_1 * 0.0000002 +\
        pred_2 * 0.0000001

# ------------------------------- hb
    
    main_weights = [
        0.0000001, 
        0.0000002, 
        0.0000004,
        0.0000005,
        0.0000011, 
        0.9999977, 
    ]

    asc_desc_weights = [0.70, 0.30]
    
    correct_weights = [ 11, 4, 0, -2, -5, -8 ]
    # #     +0.00000010,+0.00000007,+0.00000004,
    # #     -0.00000004,-0.00000007,-0.00000010
    # # ] 
    correct_weights = [cw/100_000_000 for cw in correct_weights]

       
    preds = [
        { 'wts':main_weights[0],'pred':pred_1,'res':0 },
        { 'wts':main_weights[1],'pred':pred_2,'res':0 },
        { 'wts':main_weights[2],'pred':pred_4,'res':0 },
        { 'wts':main_weights[3],'pred':pred_5,'res':0 },
        { 'wts':main_weights[4],'pred':pred_6,'res':0 },
        { 'wts':main_weights[5],'pred':pred_7,'res':0 },
    ]
    
    ascs  = sorted(copy.deepcopy(preds), key=lambda _:_['pred'],reverse=False)
    descs = sorted(copy.deepcopy(preds), key=lambda _:_['pred'],reverse=True)
    # --------------------------------------------------------------------------
    print("asc:",'\n')
    for asc  in ascs:  print(asc)
    print('\n',"desc:",'\n')
    for desc in descs: print(desc)
    print("-"*21,'\n')
    # ==========================================================================

    print('\n\n\t\t\t CORRECT.. \n\n')
    
    for asc, c_wts in zip(ascs,  correct_weights): 
        asc ['res'] = asc ['pred'] * (asc ['wts'] +c_wts)  
    for desc,c_wts in zip(descs, correct_weights): 
        desc['res'] = desc['pred'] * (desc['wts'] +c_wts)
    # --------------------------------------------------------------------------
    print("asc:",'\n')
    for asc  in ascs:  print(asc)
    print('\n',"desc:",'\n')
    for desc in descs: print(desc)
    print("="*21,'\n')
    # ==========================================================================

    print('\n\n\t\t\t RESULTS ASC/DESC.. \n\n')
    
    result_asc  = sum([asc ['res'] for asc  in ascs ])
    result_desc = sum([desc['res'] for desc in descs])
    # --------------------------------------------------------------------------
    print(f"asc={result_asc},  desc={result_desc}", "\n")
    # ==========================================================================
    
    h_blend =\
        result_asc  * asc_desc_weights[0] +\
        result_desc * asc_desc_weights[1]
    
    print (f"h-blend={h_blend}", "\n")
    # ==========================================================================

    print(f'Model 1 = {pred_1}')
    print(f'Model 2 = {pred_2}')
    print(f'Model 4 = {pred_4}')
    print(f'Model 5 = {pred_5}')
    print(f'Model 6 = {pred_6}')
    print(f'Model 7 = {pred_7}')
    
    print(f'==================== previus.pred = {pred}')
    print(f'==================== h-blend.pred = {h_blend}\n')
    
    if h_blend < 0: return pred  # LB = 17.396  
    
    return h_blend


## inference

In [ ]:
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))